# AST Hacks in
![](python-logo.png)

In [26]:
import ast
tree = ast.parse('x = 3; print(x)')
tree

In [27]:
code = compile(tree, filename='', mode='exec')
code

<code object <module> at 0x7f0028399030, file "", line 1>

In [15]:
exec(code)

3


# An AST is just a tree with nodes.

In [28]:
ast.dump(tree)

"Module(body=[Assign(targets=[Name(id='x', ctx=Store())], value=Num(n=3)), Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[]))])"

In [17]:
tree.body

[<_ast.Assign at 0x7f00283f1358>, <_ast.Expr at 0x7f0028400518>]

In [31]:
for node in ast.walk(tree):
    print(node)

In [33]:
tree.body[0] = ast.Assign(targets=[ast.Name(id='x', ctx=ast.Store())], value=ast.Num(n=5))

In [20]:
ast.dump(tree)

"Module(body=[Assign(targets=[Name(id='x', ctx=Store())], value=Num(n=5)), Expr(value=Call(func=Name(id='print', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[]))])"

In [34]:
exec(compile(tree, filename='', mode='exec'))

TypeError: required field "lineno" missing from stmt

- To compile an AST, every node must have `lineno` and `col_offset` attributes. Nodes produced by parsing regular code already have these, but nodes you create programmatically don’t. We'll use `ast.fix_missing_locations` to fix this.

In [35]:
exec(compile(ast.fix_missing_locations(tree), filename='', mode='exec'))

5


In [53]:
class RewriteName(ast.NodeTransformer):

    def visit_Call(self, node):
        return ast.copy_location(
            ast.Expr(
                value=ast.Call(func=ast.Name(id='dontprint', ctx=ast.Load()),
                args=[ast.Name(id='x', ctx=ast.Load())],
                keywords=[ast.keyword(arg='somethingelse', value=ast.Name(id='val', ctx=ast.Load()))],
                starargs=ast.Name(id='args', ctx=ast.Load()),     # gone in 3.5
                kwargs=ast.Name(id='kwargs', ctx=ast.Load()))     # gone in 3.5
            ), node)
  
tree = RewriteName().visit(tree)